In [1]:
import pandas as pd

df = pd.read_csv("../../data/produits_utf8.csv", sep=";")
df_danger = pd.read_csv("../../data/produits_classe_et_mention_danger_utf8.csv", sep=";")

In [3]:
df_danger = df_danger.drop(columns=["Unnamed: 4"])

df_danger= df_danger.rename(columns={
    "numero AMM": "amm",
    "nom produit" : "Nom_produit",
    "Libellé court": "Libellé_court",
    "Libelle long": "Toxicite_produit"
      })

In [4]:
df = df.drop(columns=["type produit"])
df = df.drop(columns=["Unnamed: 18"])

df= df.rename(columns={
    "numero AMM": "amm",
    "nom produit" : "Nom_produit",
    "seconds noms commerciaux" : "Second_noms_commerciaux",
    "type commercial" : "Type_commercial",
    "gamme usage" : "Gamme_usage",
    "mentions autorisees" : "Mentions_autorisees",
    "restrictions usage" : "Restrictions_usage", 
    "restrictions usage libelle" : "Restrictions_usage_libelle",
    "Substances actives" : "Substances_actives",
    "fonctions" : "Fonctions",
    "formulations" : "Formulations",
    "Etat d'autorisation" : "Etat_d_autorisation",
    "Date de retrait du produit" : "Date_de_retrait",
    "Date de première autorisation" : "Date_première_autorisation",
    "Numéro AMM du produit de référence" : "Numero_AMM_reference",
    "Nom du produit de référence" : "Nom_produit_reference"
      })

In [7]:
def parse_date_safe(date_str):
    try:
        return pd.to_datetime(date_str, dayfirst=True).date()
    except Exception:
        return None

df["Date_première_autorisation"] = df["Date_première_autorisation"].apply(parse_date_safe)
df["Date_de_retrait"] = df["Date_de_retrait"].apply(parse_date_safe)
df = df.where(pd.notnull(df), None)

In [13]:
df['amm'] = pd.to_numeric(df['amm'], errors='coerce')  
df = df.dropna(subset=['amm'])                         
df['amm'] = df['amm'].astype(int)                   


In [15]:
for col in df.select_dtypes(include="object").columns:
    max_len = df[col].astype(str).str.len().max()
    print(f"{col} : VARCHAR({max_len})")


Nom_produit : VARCHAR(50)
Second_noms_commerciaux : VARCHAR(241)
titulaire : VARCHAR(79)
Type_commercial : VARCHAR(20)
Gamme_usage : VARCHAR(42)
Mentions_autorisees : VARCHAR(121)
Restrictions_usage : VARCHAR(58)
Restrictions_usage_libelle : VARCHAR(510)
Substances_actives : VARCHAR(490)
Fonctions : VARCHAR(157)
Formulations : VARCHAR(69)
Etat d’autorisation : VARCHAR(8)
Date_de_retrait : VARCHAR(10)
Date_première_autorisation : VARCHAR(10)
Numero_AMM_reference : VARCHAR(27)
Nom_produit_reference : VARCHAR(87)


In [16]:
for col in df.select_dtypes(include="int").columns:
    max_val = df[col].max()
    min_val = df[col].min()
    max_digits = max(len(str(abs(max_val))), len(str(abs(min_val))))
    print(f"{col} : INTEGER (min={min_val}, max={max_val}, digits={max_digits})")


amm : INTEGER (min=1000011, max=9910016, digits=7)


In [10]:
for col in df.select_dtypes(include="float").columns:
    max_val = df[col].max()
    min_val = df[col].min()

    def count_digits(val):
        if pd.isna(val):
            return (0, 0)
        parts = f"{val:.10f}".rstrip("0").split(".")
        return (len(parts[0]), len(parts[1]) if len(parts) > 1 else 0)

    entiers, decimales = zip(*df[col].dropna().map(count_digits))

    print(f"{col} : NUMERIC({max(entiers)+max(decimales)}, {max(decimales)}) "
          f"(min={min_val}, max={max_val})")


In [20]:
df_danger= df_danger.rename(columns={
    "numero AMM": "amm",
    "nom produit" : "Nom_produit",
    "Libellé court": "Libellé_court",
    "Libelle long": "Toxicite_produit"
      })

In [21]:
for col in df_danger.select_dtypes(include="object").columns:
    max_len = df_danger[col].astype(str).str.len().max()
    print(f"{col} : VARCHAR({max_len})")


Nom_produit : VARCHAR(42)
Libellé_court : VARCHAR(18)
Libellé long : VARCHAR(134)


In [22]:
for col in df_danger.select_dtypes(include="int").columns:
    max_val = df_danger[col].max()
    min_val = df_danger[col].min()
    max_digits = max(len(str(abs(max_val))), len(str(abs(min_val))))
    print(f"{col} : INTEGER (min={min_val}, max={max_val}, digits={max_digits})")


amm : INTEGER (min=2000001, max=9900446, digits=7)
